In [143]:
#Naive Bayes Implemenetation
import os                        
import pandas as pd              
import re                        
import numpy as np                
from sklearn.metrics import confusion_matrix
import random
import nltk
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB

In [144]:
patient_df = pd.read_csv("/Users/nikhil/pre_processed_patient_diagnostics.csv")
print(patient_df.head()) # this will print the head of data_domain 
patient_df.columns #to print column names

   Unnamed: 0  Case  Age                                       Text Details  \
0           0   1.0   67  reviewed pleasant lady private outpatient clin...   
1           1   1.0   67  reviewed pleasant lady private outpatient clin...   
2           2   1.0   67  discussed finding mr b today flexible cystosco...   
3           3   2.0   81  thank referring pleasant gentleman private out...   
4           4   2.0   81  writing confirm named patient underwent succes...   

  Surgery - Yes OR No  word_count  
0                   Y          90  
1                   Y          81  
2                   N          26  
3                   N         105  
4                   Y          76  


Index(['Unnamed: 0', 'Case', 'Age', 'Text Details', 'Surgery - Yes OR No',
       'word_count'],
      dtype='object')

In [145]:
patient_df['Surgery - Yes OR No'] = patient_df['Surgery - Yes OR No'].astype(str) # taking the target column in seperate variable. we will add this to final data frame when pre processing is done
data_desc = patient_df["Text Details"] 

In [146]:
for i in range(1,len(data_desc)):
    data_desc[i] = re.sub(" \d+", " ",data_desc[i]) # this will retain only the words

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [147]:
# create the count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cnt_vct = CountVectorizer(analyzer='word',binary =True,token_pattern=r'\b\w\w+\b',
                             lowercase = True,ngram_range=(1,1),stop_words='english',encoding = 'ISO-8859-1',min_df=3)

In [148]:
# creating the document term matrix
data_mat = cnt_vct.fit_transform(data_desc)
#print(data_mat)
dense = data_mat.todense()


In [149]:
# extracting the features and storing it in a different dataset
featureset = pd.DataFrame(dense,columns=cnt_vct.get_feature_names())
featureset["Surgery - Yes OR No"] = patient_df['Surgery - Yes OR No']
classlabel = "Surgery - Yes OR No"


print(featureset.shape)

(117, 409)


In [150]:
# shuffing the data to avoid baising 
featureset = featureset.sample(frac = 1)
print(featureset.shape)
Targetindex = featureset.columns.get_loc("Surgery - Yes OR No")
np.random.seed(0)

(117, 409)


In [151]:
def Average(lst):
    return sum(lst) / len(lst)

In [195]:
#Test split out of training (Divided training into test and train sets respectively)
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
def splitter(trials,data,size):
    
    model_acc=[]
    model_f1=[]
    model_recall=[]
    model_precision=[]
    
  
    for i in range(0,trials):
        dataset_train, dataset_test = train_test_split(data, test_size=size)
        dataset_train = pd.DataFrame(dataset_train)
        dataset_test = pd.DataFrame(dataset_test)
        dataset_train.columns = featureset.columns
        dataset_test.columns = featureset.columns
        Targetindex = dataset_train.columns.get_loc("Surgery - Yes OR No")
        
        X = dataset_train.drop(dataset_train.columns[[Targetindex]], axis=1) # all the features except the target variable
        y = dataset_train["Surgery - Yes OR No"] # only the target variable
        X1 = dataset_test.drop(dataset_test.columns[[Targetindex]], axis=1) # all the features except the target variable
        y1 = dataset_test["Surgery - Yes OR No"] # only the target variable
        
        # label encode the target variable 
        encoder = preprocessing.LabelEncoder()
        train_y = encoder.fit_transform(y)
        valid_y = encoder.fit_transform(y1)
        
        clf = MultinomialNB().fit(X, train_y)
        
        testpred = clf.predict(X1)
        
        model_acc.append(accuracy_score(valid_y,testpred))
        model_f1.append(f1_score(valid_y,testpred,pos_label=1,average="binary"))
        model_recall.append(recall_score(valid_y,testpred,pos_label=1,average="binary"))
        model_precision.append(precision_score(valid_y,testpred,pos_label=1,average="binary"))
        
        matrix=confusion_matrix(valid_y,testpred)
        print("Confusion matrix for split",i)
        print(matrix)
        
    print("Average metrics for Naive Bayes are:")
    
    acc_avg=Average(model_acc)
    f1_avg=Average(model_f1)
    recall_avg=Average(model_recall)
    precision_avg=Average(model_precision)

    return acc_avg,f1_avg,recall_avg,precision_avg



In [196]:
splitter(10,np.array(featureset),0.3)

Confusion matrix for split 0
[[ 4  2]
 [ 1 29]]
Confusion matrix for split 1
[[ 3  1]
 [ 3 29]]
Confusion matrix for split 2
[[ 2  3]
 [ 3 28]]
Confusion matrix for split 3
[[ 3  5]
 [ 1 27]]
Confusion matrix for split 4
[[ 4  1]
 [ 2 29]]
Confusion matrix for split 5
[[ 3  4]
 [ 4 25]]
Confusion matrix for split 6
[[ 3  2]
 [ 3 28]]
Confusion matrix for split 7
[[ 3  4]
 [ 4 25]]
Confusion matrix for split 8
[[ 3  2]
 [ 1 30]]
Confusion matrix for split 9
[[ 3  1]
 [ 1 31]]
Average metrics for Naive Bayes are:


(0.8666666666666668, 0.920365069198233, 0.9239767731341703, 0.9179514275120504)